# Module 01: Experiment Tracking with MLflow

**Difficulty**: ⭐⭐ Intermediate
**Estimated Time**: 60 minutes
**Prerequisites**: 
- [Module 00: Introduction to MLOps](00_introduction_to_mlops.ipynb)
- Basic understanding of scikit-learn

## Learning Objectives
By the end of this notebook, you will be able to:
1. Set up MLflow for experiment tracking
2. Log parameters, metrics, and artifacts to MLflow
3. Compare multiple experiments using MLflow UI
4. Use MLflow autologging for popular ML frameworks
5. Organize experiments with tags and nested runs

## 1. Why Experiment Tracking?

### The Problem Without Tracking

Imagine you've trained 50 models over 2 weeks:
- **Model #32 had the best validation accuracy**, but which hyperparameters did you use?
- **Model #47 worked great on production-like data**, but what was the random seed?
- **Your manager asks**: "Can you reproduce the model from last Tuesday?"
- **Result**: 😰 You have no idea!

### The Solution: MLflow Tracking

**MLflow** is an open-source platform for managing the ML lifecycle. It has four main components:

1. **MLflow Tracking**: Log and query experiments (parameters, metrics, artifacts)
2. **MLflow Projects**: Package code in reproducible format
3. **MLflow Models**: Deploy models to various platforms
4. **MLflow Registry**: Central model store for versioning and staging

In this module, we focus on **MLflow Tracking**.

In [ ]:
# Setup: Import all required libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.datasets import make_classification, load_diabetes
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score
)

# MLflow
import mlflow
import mlflow.sklearn

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print(f"MLflow version: {mlflow.__version__}")

## 2. MLflow Basics: Your First Tracked Experiment

Let's start with a simple example to understand MLflow's core concepts.

In [ ]:
# Set up MLflow tracking URI (local directory for this tutorial)
# In production, this would point to a remote tracking server
mlflow.set_tracking_uri("file:./mlruns")

# Set experiment name
# All runs will be grouped under this experiment
mlflow.set_experiment("classification_experiments")

print("MLflow tracking URI:", mlflow.get_tracking_uri())
print("Current experiment:", mlflow.get_experiment_by_name("classification_experiments").name)

In [ ]:
# Generate synthetic dataset for classification
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    random_state=42
)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

### Basic MLflow Run Structure

Every tracked experiment follows this pattern:

```python
with mlflow.start_run():
    # 1. Log parameters
    mlflow.log_param("param_name", value)
    
    # 2. Train model
    model.fit(X_train, y_train)
    
    # 3. Log metrics
    mlflow.log_metric("accuracy", accuracy)
    
    # 4. Log model
    mlflow.sklearn.log_model(model, "model")
```

In [ ]:
# Example 1: Track a simple logistic regression model

with mlflow.start_run(run_name="logistic_regression_baseline"):
    
    # Define hyperparameters
    max_iter = 1000
    C = 1.0
    
    # Log parameters
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("C", C)
    mlflow.log_param("solver", "lbfgs")
    
    # Train model
    model = LogisticRegression(max_iter=max_iter, C=C, random_state=42)
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log the trained model
    mlflow.sklearn.log_model(model, "model")
    
    # Add tags for better organization
    mlflow.set_tag("stage", "baseline")
    mlflow.set_tag("team", "data-science")
    
    print(f"✓ Logged run with accuracy: {accuracy:.4f}")
    print(f"  Run ID: {mlflow.active_run().info.run_id}")

## 3. Comparing Multiple Experiments

The power of MLflow comes from comparing different approaches systematically.

In [ ]:
# Experiment with different Random Forest configurations

# Define hyperparameter grid
rf_configs = [
    {"n_estimators": 50, "max_depth": 5, "min_samples_split": 2},
    {"n_estimators": 100, "max_depth": 10, "min_samples_split": 2},
    {"n_estimators": 100, "max_depth": 10, "min_samples_split": 5},
    {"n_estimators": 200, "max_depth": 15, "min_samples_split": 2},
    {"n_estimators": 200, "max_depth": None, "min_samples_split": 5},
]

results = []

for i, config in enumerate(rf_configs, 1):
    with mlflow.start_run(run_name=f"random_forest_config_{i}"):
        
        # Log all hyperparameters
        mlflow.log_param("model_type", "RandomForest")
        mlflow.log_params(config)  # Log dictionary of params at once
        
        # Train model
        model = RandomForestClassifier(**config, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)
        
        # Predictions
        y_pred = model.predict(X_test)
        
        # Metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Log metrics
        mlflow.log_metrics({
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        })
        
        # Log model
        mlflow.sklearn.log_model(model, "model")
        
        # Tags
        mlflow.set_tag("stage", "experimentation")
        mlflow.set_tag("model_family", "tree_based")
        
        # Store results for visualization
        results.append({
            "config_id": i,
            "n_estimators": config["n_estimators"],
            "max_depth": config["max_depth"] or "None",
            "accuracy": accuracy,
            "f1_score": f1
        })
        
        print(f"✓ Config {i}: Accuracy={accuracy:.4f}, F1={f1:.4f}")

results_df = pd.DataFrame(results)
print("\nAll experiments completed!")

In [ ]:
# Visualize experiment results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Accuracy comparison
axes[0].bar(results_df['config_id'], results_df['accuracy'], 
            color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Configuration ID', fontweight='bold')
axes[0].set_ylabel('Accuracy', fontweight='bold')
axes[0].set_title('Model Accuracy Across Configurations', fontweight='bold')
axes[0].set_ylim(0.85, 0.95)
axes[0].grid(axis='y', alpha=0.3)

# Plot 2: F1 Score comparison
axes[1].bar(results_df['config_id'], results_df['f1_score'], 
            color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Configuration ID', fontweight='bold')
axes[1].set_ylabel('F1 Score', fontweight='bold')
axes[1].set_title('F1 Score Across Configurations', fontweight='bold')
axes[1].set_ylim(0.85, 0.95)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Show best configuration
best_config = results_df.loc[results_df['f1_score'].idxmax()]
print(f"\nBest Configuration: #{int(best_config['config_id'])}")
print(f"  n_estimators: {int(best_config['n_estimators'])}")
print(f"  max_depth: {best_config['max_depth']}")
print(f"  F1 Score: {best_config['f1_score']:.4f}")

## 4. Logging Artifacts (Plots, Files, Models)

Beyond metrics, you can log:
- **Plots**: Confusion matrices, ROC curves, feature importance
- **Files**: Datasets, configuration files, text reports
- **Models**: Serialized models for later use

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tempfile
import os

with mlflow.start_run(run_name="random_forest_with_artifacts"):
    
    # Train model
    config = {"n_estimators": 100, "max_depth": 10, "random_state": 42}
    mlflow.log_params(config)
    
    model = RandomForestClassifier(**config)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Calculate and log metrics
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    
    # Create and log confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    disp.plot(ax=ax, cmap='Blues', values_format='d')
    ax.set_title('Confusion Matrix', fontweight='bold', fontsize=14)
    
    # Save plot temporarily and log to MLflow
    with tempfile.TemporaryDirectory() as tmpdir:
        plot_path = os.path.join(tmpdir, "confusion_matrix.png")
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        mlflow.log_artifact(plot_path, artifact_path="plots")
    
    plt.close()
    
    # Create and log feature importance plot
    feature_importance = pd.DataFrame({
        'feature': [f'feature_{i}' for i in range(X.shape[1])],
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False).head(10)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.barh(feature_importance['feature'], feature_importance['importance'], color='teal')
    ax.set_xlabel('Importance', fontweight='bold')
    ax.set_title('Top 10 Feature Importances', fontweight='bold', fontsize=14)
    ax.invert_yaxis()
    
    with tempfile.TemporaryDirectory() as tmpdir:
        plot_path = os.path.join(tmpdir, "feature_importance.png")
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        mlflow.log_artifact(plot_path, artifact_path="plots")
    
    plt.close()
    
    # Log text report as artifact
    report = f"""
    Model Performance Report
    ========================
    Model Type: Random Forest
    Accuracy: {accuracy:.4f}
    
    Hyperparameters:
    - n_estimators: {config['n_estimators']}
    - max_depth: {config['max_depth']}
    
    Training set size: {len(X_train)}
    Test set size: {len(X_test)}
    """
    
    with tempfile.TemporaryDirectory() as tmpdir:
        report_path = os.path.join(tmpdir, "report.txt")
        with open(report_path, 'w') as f:
            f.write(report)
        mlflow.log_artifact(report_path, artifact_path="reports")
    
    # Log model
    mlflow.sklearn.log_model(model, "model")
    
    print("✓ Logged model with plots and reports!")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Artifacts: confusion_matrix.png, feature_importance.png, report.txt")

## 5. MLflow Autologging

MLflow provides **autologging** for popular frameworks:
- scikit-learn
- TensorFlow/Keras
- PyTorch
- XGBoost
- LightGBM

**Autologging automatically captures**:
- Model hyperparameters
- Training metrics
- Model artifacts
- Plots (for some frameworks)

In [ ]:
# Enable autologging for scikit-learn
mlflow.sklearn.autolog()

with mlflow.start_run(run_name="autolog_random_forest"):
    # Just train the model - MLflow logs everything automatically!
    model = RandomForestClassifier(
        n_estimators=150,
        max_depth=12,
        min_samples_split=3,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Evaluate (autolog captures this too)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # You can still log custom metrics/tags
    mlflow.log_metric("custom_metric", accuracy * 100)
    mlflow.set_tag("note", "Using autologging")
    
    print(f"✓ Autologging captured everything automatically!")
    print(f"  Check MLflow UI to see all logged parameters and metrics")

# Disable autologging when done
mlflow.sklearn.autolog(disable=True)

## 6. Nested Runs for Complex Experiments

For experiments with multiple stages (e.g., hyperparameter tuning with cross-validation), use **nested runs**.

In [ ]:
# Nested runs example: Parent run with child runs for each fold

with mlflow.start_run(run_name="cross_validation_experiment") as parent_run:
    
    # Log parent-level info
    mlflow.log_param("cv_folds", 5)
    mlflow.log_param("model_type", "RandomForest")
    
    # Configuration
    config = {"n_estimators": 100, "max_depth": 10, "random_state": 42}
    mlflow.log_params(config)
    
    # Perform cross-validation with nested runs
    model = RandomForestClassifier(**config)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    # Log each fold as a nested run
    for fold, score in enumerate(cv_scores, 1):
        with mlflow.start_run(run_name=f"fold_{fold}", nested=True):
            mlflow.log_metric("accuracy", score)
            mlflow.log_param("fold", fold)
    
    # Log aggregate metrics in parent run
    mlflow.log_metric("mean_cv_accuracy", cv_scores.mean())
    mlflow.log_metric("std_cv_accuracy", cv_scores.std())
    
    print(f"✓ Cross-validation completed")
    print(f"  Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
    print(f"  Parent Run ID: {parent_run.info.run_id}")

## 7. Querying and Searching Experiments

MLflow provides a powerful API to search and filter runs.

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Get current experiment
experiment = client.get_experiment_by_name("classification_experiments")
experiment_id = experiment.experiment_id

# Search for runs with accuracy > 0.90
runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="metrics.accuracy > 0.90",
    order_by=["metrics.accuracy DESC"],
    max_results=10
)

print(f"Found {len(runs)} runs with accuracy > 0.90\n")

# Display top runs
results = []
for run in runs:
    results.append({
        "run_name": run.data.tags.get("mlflow.runName", "N/A"),
        "accuracy": run.data.metrics.get("accuracy", 0),
        "model_type": run.data.params.get("model_type", "N/A"),
        "run_id": run.info.run_id[:8]  # Shortened for display
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

## 8. Accessing the MLflow UI

To view all your experiments in a nice web interface:

```bash
# Run this command in your terminal (not in notebook)
mlflow ui --backend-store-uri file:./mlruns
```

Then open your browser to: **http://localhost:5000**

### What You Can Do in the UI:
- **Compare runs** side-by-side
- **Visualize metrics** over time
- **Download artifacts** (models, plots, reports)
- **Search and filter** runs
- **Add notes and tags**
- **Promote models** to registry

## 9. Exercises

Test your understanding with these exercises.

### Exercise 1: Track a Regression Model

Use the diabetes dataset and track a regression experiment with MLflow.

**Requirements**:
1. Load diabetes dataset
2. Split into train/test
3. Train a GradientBoostingRegressor
4. Log parameters: n_estimators, learning_rate, max_depth
5. Log metrics: MSE, MAE, R²
6. Log the trained model

In [ ]:
# Exercise 1: Your code here

# Load dataset
# YOUR CODE HERE

# Split data
# YOUR CODE HERE

# Train and log with MLflow
# YOUR CODE HERE

In [ ]:
# Exercise 1 Solution

# Load diabetes dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Set experiment
mlflow.set_experiment("regression_experiments")

with mlflow.start_run(run_name="gradient_boosting_regression"):
    
    # Hyperparameters
    params = {
        "n_estimators": 100,
        "learning_rate": 0.1,
        "max_depth": 3,
        "random_state": 42
    }
    
    # Log parameters
    mlflow.log_params(params)
    
    # Train model
    model = GradientBoostingRegressor(**params)
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metrics({
        "mse": mse,
        "mae": mae,
        "r2_score": r2
    })
    
    # Log model
    mlflow.sklearn.log_model(model, "model")
    
    print(f"✓ Regression model tracked successfully!")
    print(f"  MSE: {mse:.2f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  R²: {r2:.4f}")

### Exercise 2: Compare Multiple Learning Rates

Run experiments with different learning rates and find the best one.

**Requirements**:
1. Test learning rates: [0.01, 0.05, 0.1, 0.2, 0.5]
2. Keep other hyperparameters constant
3. Log all experiments
4. Print which learning rate gave the best R² score

In [ ]:
# Exercise 2 Solution

learning_rates = [0.01, 0.05, 0.1, 0.2, 0.5]
results = []

for lr in learning_rates:
    with mlflow.start_run(run_name=f"gbr_lr_{lr}"):
        
        # Log parameters
        params = {
            "n_estimators": 100,
            "learning_rate": lr,
            "max_depth": 3,
            "random_state": 42
        }
        mlflow.log_params(params)
        
        # Train model
        model = GradientBoostingRegressor(**params)
        model.fit(X_train, y_train)
        
        # Evaluate
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        
        # Log metrics
        mlflow.log_metric("r2_score", r2)
        
        results.append({"learning_rate": lr, "r2_score": r2})
        print(f"Learning Rate {lr}: R² = {r2:.4f}")

# Find best
results_df = pd.DataFrame(results)
best = results_df.loc[results_df['r2_score'].idxmax()]
print(f"\n✓ Best learning rate: {best['learning_rate']} (R² = {best['r2_score']:.4f})")

### Exercise 3: Log Prediction vs Actual Plot

Create a scatter plot comparing predictions vs actual values and log it as an artifact.

**Requirements**:
1. Train a model
2. Create scatter plot (actual vs predicted)
3. Add reference line (perfect predictions)
4. Save and log plot to MLflow

In [ ]:
# Exercise 3 Solution

with mlflow.start_run(run_name="regression_with_plot"):
    
    # Train model
    params = {"n_estimators": 100, "learning_rate": 0.1, "random_state": 42}
    mlflow.log_params(params)
    
    model = GradientBoostingRegressor(**params)
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)
    
    # Metrics
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric("r2_score", r2)
    
    # Create plot
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.scatter(y_test, y_pred, alpha=0.6, edgecolors='k', s=50)
    
    # Add perfect prediction line
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    ax.set_xlabel('Actual Values', fontweight='bold', fontsize=12)
    ax.set_ylabel('Predicted Values', fontweight='bold', fontsize=12)
    ax.set_title(f'Predictions vs Actual (R² = {r2:.4f})', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Save and log plot
    with tempfile.TemporaryDirectory() as tmpdir:
        plot_path = os.path.join(tmpdir, "predictions_vs_actual.png")
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        mlflow.log_artifact(plot_path, artifact_path="plots")
    
    plt.show()
    print("✓ Plot logged to MLflow!")

## 10. Summary

### Key Takeaways

1. **MLflow Tracking** solves the "which model was that?" problem by systematically logging experiments

2. **Core tracking components**:
   - **Parameters**: Hyperparameters and configuration
   - **Metrics**: Performance measures (accuracy, loss, etc.)
   - **Artifacts**: Models, plots, reports, data files
   - **Tags**: Metadata for organization

3. **Autologging** saves time by automatically capturing model info for popular frameworks

4. **MLflow UI** provides powerful visualization and comparison tools

5. **Nested runs** help organize complex experiments like cross-validation

6. **Search API** enables programmatic filtering and analysis of experiments

### Best Practices

- Always use descriptive run names
- Log enough context to reproduce the experiment
- Use tags to organize experiments by team, stage, or project
- Log plots and reports, not just numbers
- Review experiments regularly in the UI

### What's Next?

In **Module 02**, we'll learn about:
- **Model versioning** with MLflow Model Registry
- **Staging models** (dev → staging → production)
- **Model lineage** tracking
- **Transitioning models** between stages

## 11. Additional Resources

### Documentation
- **MLflow Tracking**: https://mlflow.org/docs/latest/tracking.html
- **MLflow Python API**: https://mlflow.org/docs/latest/python_api/index.html
- **Autologging**: https://mlflow.org/docs/latest/tracking.html#automatic-logging

### Tutorials
- **MLflow Quickstart**: https://mlflow.org/docs/latest/quickstart.html
- **MLflow Examples**: https://github.com/mlflow/mlflow/tree/master/examples

### Advanced Topics
- Setting up remote tracking servers
- Integrating with cloud storage (S3, Azure Blob)
- Using MLflow with distributed training
- MLflow Projects for reproducible workflows